In [8]:
import nbaScraper as ns
import actNetScraper as ans
from datetime import datetime, timedelta

database_export = True
store_locally = True

# nba website, basketball referenece scraper
scraper = ns.scraper(
    browser_path = '..\\..\\browser\\geckodriver.exe',
    database_export = database_export, 
    store_locally = store_locally,
    pymysql_conn_str =  None
)
today = scraper.meta_data['today_dt']
lastNgames = 10
day_adj = -1

### prop line/odd scraper
leagues = None   # None or list ['nba', 'nhl', 'nfl']
specified =  []
# day adjustment from today (date of running script), negative = dates into the past
dayJump = 0 
# date can be a list of dates if multiple need scraping 'YYYY-MM-DD'
# default is to only pull today or today + dayJump
dates = [(datetime.today() + timedelta(days=dayJump)).strftime('%Y-%m-%d')]
#dates = ['2024-01-05']#, '2024-01-06', '2024-01-07', '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11']

prop_scraper = ans.actNetScraper(
    browser_path = '..\\..\\browser\\geckodriver.exe',
    dates = dates,
    leagues = leagues,
    database_export = database_export, 
    store_locally = store_locally,
    config_path = '..\\..\\..\\..\\Notes-General\\config.txt',
    second_run = False
)

# turn to False if issues loading new players
#prop_scraper.update_players = False

# update the league list to only ones with games today
#leagues = prop_scraper.check_for_league_games(date_check = None, update_class_leagues_var = True)
print('scraping for', prop_scraper.leagues, 'on', prop_scraper.dates)

scraping for ['nba'] on ['2025-02-14']


In [9]:
prop_scraper.scrape(
    sleep_secs = 3, 
    specific_props = specified,
    leagues_override = leagues,
    an_state_code = 'BC'
)

prop_scraper.processScrapes(
    remove_dups = True,
    specific_props = specified
)

if prop_scraper.scrape_error_flag:
    print(prop_scraper.scrape_errors)
    #prop_scraper.tryMissingProps()


scraping nba ...
original rows:  (70, 18)
after dups removed:  (70, 18)


KeyError: "['player', 'abbr'] not in index"

In [6]:
scraper.get_nba_team_playtype_data(
    base_url = 'https://www.nba.com/stats/teams/{playtype}?TypeGrouping={sideofball}&SeasonType={type}',
    play_types = [
        'isolation', 'transition', 'ball-handler', 'roll-man', 'playtype-post-up',
        'spot-up', 'hand-off', 'cut', 'off-screen','putbacks'
    ],
    sides = ['offensive', 'defensive'],
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    database_table = 'statsteamplaytypes'
)

scraper.get_nba_team_shotzone_data(
    base_url = 'https://www.nba.com/stats/teams/{sideOfBall}?DistanceRange=By+Zone&LastNGames={lastNgames}&SeasonType={type}&DateTo={endDate}', 
    sides = {'offensive':'shooting', 'defensive':'opponent-shooting'},
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    lastNgames = lastNgames,
    database_table = 'statsteamshotzones',
    endDate = today
)

scraper.get_nba_team_stats(
        base_url = 'https://www.nba.com/stats/teams/{stats}?SeasonType={seasonType}',
        stats = ['traditional', 'advanced', 'opponent'],
        season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        lastNgames = 0,
        database_table = 'statsteam',
        endDate = None
)

scraper.get_nba_player_playtype_data(
    base_url = 'https://www.nba.com/stats/players/{playtype}?TypeGrouping={sideofball}&SeasonType={type}', 
    play_types = [
        'isolation', 'transition', 'ball-handler', 'roll-man', 'playtype-post-up',
        'spot-up', 'hand-off', 'cut', 'off-screen','putbacks'
    ],
    sides = ['offensive'],
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    database_table = 'statsplayerplaytypes'
)

scraper.get_nba_player_shotzone_data(
    base_url = 'https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone&LastNGames={lastNgames}&SeasonType={type}&DateTo={endDate}', 
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    lastNgames = lastNgames,
    database_table = 'statsplayershotzones',
    endDate = today
)

scraper.get_nba_player_passing_data(
    base_url = 'https://www.nba.com/stats/players/passing?DateFrom={d1}&DateTo={d2}&LastNGames=0&PerMode=Totals&SeasonType={type}', 
    today_date = today,
    day_adjuster = day_adj,
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    #lastNgames = 10,
    database_table = 'statsplayerpassing'
)

scraper.get_nba_player_rebounding_data(
    base_url = 'https://www.nba.com/stats/players/rebounding?DateFrom={d1}&DateTo={d2}&LastNGames=0&PerMode=Totals&SeasonType={type}', 
    today_date = today,
    day_adjuster = day_adj,
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    #lastNgames = 10,
    database_table = 'statsplayerrebounding'
)

scraper.get_bref_pos_estimates(
        base_url = 'https://www.basketball-reference.com/teams/{team}/{season}.html#pbp', 
        today_date = today,
        season = 2025,
        database_table = 'brefmisc'
)

if scraper.scrape_error_flag:
    print(scraper.scrape_errors)

nba team play type scraped...
nba team shot zone scraped...
nba team stats scraped...
nba player play type scraped...
nba player shot zone scraped...
nba player passing scraped...
nba player rebounding scraped...
bref player pos estimates scraped...
{'statsteamplaytypes': {'url': [], 'db': []}, 'statsteamshotzones': {'url': [[{'defensive': 'opponent-shooting'}, '10', 'Regular+Season']], 'db': []}, 'statsteam': {'url': [], 'db': []}, 'statsplayerplaytypes': {'url': [], 'db': []}, 'statsplayershotzones': {'url': [], 'db': []}, 'statsplayerpassing': {'url': [], 'db': []}, 'statsplayerrebounding': {'url': [], 'db': []}, 'brefmisc': {'url': [], 'db': []}}


In [7]:

scraper.get_nba_team_shotzone_data(
    base_url = 'https://www.nba.com/stats/teams/{sideOfBall}?DistanceRange=By+Zone&LastNGames={lastNgames}&SeasonType={type}&DateTo={endDate}', 
    sides = {'defensive':'opponent-shooting'},
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    lastNgames = lastNgames,
    database_table = 'statsteamshotzones',
    endDate = today
)


nba team shot zone scraped...


In [ ]:
#prop_scraper.tryMissingProps()
# this didn't work as expected, 
#### added SOG to the nba scrape_errors
#### added new key-value pair for the missing prop:  'missing_dates': ['sog', '2025-01-02', 0]

In [ ]:
prop_scraper.scrape_errors

In [7]:
missing = prop_scraper.scrape_errors
del missing['nfl']
#prop_scraper.scrape_errors

In [ ]:
# scrape missed props
import random
wait_secs = random.randint(1,5)

for k, v in missing.items():

    if len(v['missing_props']) > 0:
        leagues_with_missing_props = [k]
        missing_props = v['missing_props']
        print('rescraping', k, 'for', missing_props)

        prop_scraper_missing = ans.actNetScraper(
            browser_path = '..\\..\\browser\\geckodriver.exe',
            dates = dates,
            leagues = leagues_with_missing_props,
            database_export = database_export, 
            store_locally = store_locally,
            config_path = '..\\..\\..\\..\\Notes-General\\config.txt',
            second_run=True
        )

        prop_scraper_missing.scrape(
            sleep_secs = wait_secs, 
            specific_props = missing_props,
            leagues_override = None
        )

        prop_scraper_missing.processScrapes(
            remove_dups = True,
            specific_props = missing_props
        )

# scratch